In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolorsx
from matplotlib.patches import Circle

import os

from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import proj_plane_pixel_scales
from astropy.nddata import Cutout2D

from reproject import reproject_exact

from AstroColour.AstroColour import RGB

import matplotlib.patheffects as patheffects

%matplotlib widget

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
fig_width_pt = 244.0  # Get this from LaTeX using \the\columnwidth
text_width_pt = 508.0 # Get this from LaTeX using \the\textwidth

# plt.rc('font', size=21)
# plt.rc('xtick', labelsize=21)
# plt.rc('ytick', labelsize=21)

inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt*1.5 # width in inches
fig_width_full = text_width_pt*inches_per_pt  # 17

In [ ]:
files = ['g', 'r', 'i', 'z']
f_files = []
wcs_list = []

ra = 233.819485943 
dec = 30.8654861098

i = 0
for f in files:
    file = f'/Users/zgl12/rings.v3.skycell.1981.071.stk.{f}.unconv.fits'
    hdu = fits.open(file)

    image_data = hdu[1].data
    wcs = WCS(hdu[1].header)
    hdu.close()
    
    # Convert RA/Dec to pixel coords (x, y)
    x, y = wcs.all_world2pix(ra, dec, 0)

    # Cutout around that position
    size = (530, 530)  # (ny, nx)
    cutout = Cutout2D(image_data, (x, y), size, wcs=wcs)

    cut_data = cutout.data
    cut_wcs = cutout.wcs
    
    wcs_list.append(cut_wcs)
    
    f_files.append(cut_data)

In [ ]:
for i in range(len(f_files)):

    plt.figure()
    plt.subplot(projection=wcs_list[i])
    plt.imshow(f_files[i], origin='lower', cmap='gray')
    plt.grid(color='white', ls='solid')
    plt.xlim(35, 565)
    plt.ylim(35, 565)
    plt.show()

In [ ]:
# hdu_cut = fits.PrimaryHDU(data=f_files[1], header=wcs_list[1].to_header())
# hdu_cut.writeto(f'cutout_r.fits', overwrite=True)
# hdu.close()

In [ ]:
hdu = fits.open('cutout_r.fits')
header = hdu[0].header
data = hdu[0].data
wcs = WCS(header)
hdu.close()

In [ ]:
wcs

In [ ]:
x, y = wcs.all_world2pix(ra, dec, 0)

plt.figure(figsize = (6,6))
plt.imshow(f_files[0], origin = 'lower')
plt.scatter(x, y, s = 100, edgecolor = 'red', facecolor = 'none')
plt.show()

In [ ]:
rgb = RGB(f_files,
          save = False, save_name = 'sn2025ngs', save_folder = '/Users/zgl12/', 
          epsf_plot=False, epsf = True,
          bkg_plot = False, temp_save = True, run = False, manual_override=35)

In [ ]:
colour = rgb.master_plot(f_files, 
                         colours = ['blue', 'green', 'red', 'red'], 
                         intensities = [0.57, 1, 0.25, 0.25], 
                         gamma = [0.57, 0.57, 0.57, 0.57],
                         norms = ['sinh', 'sinh', 'sinh', 'sinh'], 
                         uppers = [99, 99, 99, 99],
                         lowers = [15, 15, 15, 15], 
                         interactive=False)